Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155: Assignment 7

Topics Covered: References and Side Effects.

__Name__: WRITE YOUR NAME HERE

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## P1 :  Functional Programming Finite State Machines.

In this assignment, we understand how to program a finite state machines (FSM). A FSM has the following components:
  - A set of states $Q: \{s_1, \ldots, s_n \}$.
  - A set of input alphabet: $\Sigma: \{ a_1, \ldots, a_m \}$.
  - A transition function from each state, given an input to a new state: $ \delta: Q \times \Sigma \rightarrow Q$ wherein $\delta(s_j, a_k)$ is a state $s_l$ that is reached when the alphabet $a_k$ is encountered in state $s_j$.
  - An initial state $s_0 \in Q$ and 
  - A set of accepting states $F \subseteq Q$.

If you have never encountered an FSM or its definition in the past (ouch!), then please look up these notes online: http://infolab.stanford.edu/~ullman/ialc/spr10/slides/fa2.pdf

Let us implement FSMs using mutables complete the missing functions.

In [2]:
type StateID = Int
type Alph = Int

class MutableFSM(val delta: StateID => Alph => StateID,
          val init_state_id: StateID, 
          val is_accepting: StateID => Boolean){
    
    /*-- Maintain the ID of the current state --*/
    var current_state_id : StateID = init_state_id
    /*-- A getter for the current state --*/
    def get_current_state: StateID = { current_state_id }
    /*-- Is the current state accepting? --*/
    def accepting: Boolean =  { is_accepting(current_state_id) }
    /*-- Apply transition from current state for alphabet `alph`: use the function delta provided as a class parameter.--*/
    def step(alph: Int):Unit ={
        // YOUR CODE HERE
       current_state_id = delta(current_state_id)(alph);
    }
    /*-- multiple_steps on a list of alphabets
    Use of for-loops ALLOWED for this one problem!
    Use the function step that you just implemented.
    --*/
    def multiple_steps(alph_seq: List[Int]): Unit = {
        // YOUR CODE HERE
        for(x <- alph_seq)
        {
            step(x)
        }
    }
}

defined type StateID
defined type Alph
defined class MutableFSM

In [3]:
//BEGIN TESTS
def delta (st: StateID) (alph: Alph) = (st, alph) match {
    case (0, 0) => 1
    case (0, 1) => 0
    case (0, 2) => 1
    case (1, 0) => 1
    case (1, 1) => 1
    case (1, 2) => 2
    case (2, 1) => 0
    case (2, 0) => 1
    case (2, 2) => 2
}

def is_accepting(st: StateID) = { st == 2 }

val mfsm = new MutableFSM(delta, 0, is_accepting)

mfsm.step(0)
assert (mfsm.get_current_state == 1, s"A: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"B: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"C: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"D: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(0)
assert (mfsm.get_current_state == 1, s"E: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"F: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 1, s"G: expected state: 1, your code returns: ${mfsm.get_current_state}")
mfsm.step(2)
assert (mfsm.get_current_state == 2, s"H: expected state: 2, your code returns: ${mfsm.get_current_state}")
mfsm.step(1)
assert (mfsm.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
mfsm.multiple_steps(List(0,1,2,2,0,1,1,2,1))
assert (mfsm.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
passed(5)
//END TESTS


*** Tests Passed (5 points) ***


defined function delta
defined function is_accepting
mfsm: MutableFSM = ammonite.$sess.cmd1$Helper$MutableFSM@1d328942

Let us now explore a functional style of implementing a FSM.
Note the following changes:
  - `current_state_id` is immutable now.
  - The function `step` returns a new FSM whose current state must be set to the next state from current state and alphabet.
  - Similarly the function `multiple_step` returns a new FSM structure. 
  
 __Restrictions__  For the problem, below you are not allowed to use `var` or loops.  Please use `foldLeft` to implement `multiple_step` function.

In [4]:
class FSM(val current_state_id: StateID,
          val delta: StateID => Alph => StateID,
          val is_accepting: StateID => Boolean) {
    /*-- A getter for the current state --*/
    def get_current_state: Int = current_state_id
    /*-- Is the current state accepting? --*/
    def accepting: Boolean = is_accepting(current_state_id)
    /*-- Apply transition from current state for alphabet `alph`--*/
    def step(alph: Int): FSM= {
        // YOUR CODE HERE
        val fsm =  delta(current_state_id)(alph);
        val fms_Prime = new FSM(fsm, delta, is_accepting)
        fms_Prime
    }
    /*-- multiple_steps on a list of alphabets
    Use of var/loops not allowed.
    --*/
    def multiple_steps(alph_seq: List[Alph]):FSM = {
        // YOUR CODE HERE
        val init_FSM = new FSM (current_state_id, delta, is_accepting)
        alph_seq.foldLeft(init_FSM){
            (curr_FSM, elt) => curr_FSM.step(elt)
        }
        
    }
}

defined class FSM

In [5]:
//BEGIN TEST
def delta (st: StateID) (alph: Alph) = (st, alph) match {
    case (0, 0) => 1
    case (0, 1) => 0
    case (0, 2) => 1
    case (1, 0) => 1
    case (1, 1) => 1
    case (1, 2) => 2
    case (2, 1) => 0
    case (2, 0) => 1
    case (2, 2) => 2
}

def is_accepting(st: StateID) = { st == 2 }

val mfsm0 = new FSM(0, delta, is_accepting)

val mfsm1 = mfsm0.step(0)
assert (mfsm1.get_current_state == 1, s"A: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm2 = mfsm1.step(1)
assert (mfsm2.get_current_state == 1, s"B: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm3 = mfsm2.step(2)
assert (mfsm3.get_current_state == 2, s"C: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm4 = mfsm3.step(2)
assert (mfsm4.get_current_state == 2, s"D: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm5 = mfsm4.step(0)
assert (mfsm5.get_current_state == 1, s"E: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm6 = mfsm5.step(1)
assert (mfsm6.get_current_state == 1, s"F: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm7 = mfsm6.step(1)
assert (mfsm7.get_current_state == 1, s"G: expected state: 1, your code returns: ${mfsm.get_current_state}")
val mfsm8 = mfsm7.step(2)
assert (mfsm8.get_current_state == 2, s"H: expected state: 2, your code returns: ${mfsm.get_current_state}")
val mfsm9 = mfsm8.step(1)
assert (mfsm9.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
val mfsm10 = mfsm9.multiple_steps(List(0,1,2,2,0,1,1,2,1))
assert (mfsm10.get_current_state == 0, s"I: expected state: 0, your code returns: ${mfsm.get_current_state}")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function delta
defined function is_accepting
mfsm0: FSM = ammonite.$sess.cmd3$Helper$FSM@1bb038c6
mfsm1: FSM = ammonite.$sess.cmd3$Helper$FSM@1c83b5af
mfsm2: FSM = ammonite.$sess.cmd3$Helper$FSM@99fa6ec
mfsm3: FSM = ammonite.$sess.cmd3$Helper$FSM@7131176d
mfsm4: FSM = ammonite.$sess.cmd3$Helper$FSM@4df8d871
mfsm5: FSM = ammonite.$sess.cmd3$Helper$FSM@432e22ee
mfsm6: FSM = ammonite.$sess.cmd3$Helper$FSM@63715f4a
mfsm7: FSM = ammonite.$sess.cmd3$Helper$FSM@3f54c7ee
mfsm8: FSM = ammonite.$sess.cmd3$Helper$FSM@20ad11ea
mfsm9: FSM = ammonite.$sess.cmd3$Helper$FSM@4f72e262
mfsm10: FSM = ammonite.$sess.cmd3$Helper$FSM@65737614

# Problem 2: Block of Statements in Lettuce with References (20 points).

You have been using a sequence of statements in almost all languages that you have learned but for Lettuce.  If we write a sequence of expressions (statements) as follows

~~~
<expression 1> ;
<expression 2> ;
<expression 3> ;
<expression 4> ;
... 
<expression n>
~~~
Each of these expressions is interpreted in sequence starting from 1 to n. The return value of the entire
block of expressions (statements) is simply the value returned by the very last statement. 

We would like to add this to Lettuce in the form of a new production.

$$\mathbf{Expr} \ \rightarrow\ Seq(\mathbf{Expr}, \mathbf{Expr})$$

The expression $\texttt{Seq(e1, e2)}$ is a sequence `e1` followed by `e2`. The overall value of a `Seq` is that
returned by `e2`. However, since we have Lettuce with references, it is possible that `e1` may have side effects that are visible in `e2`.

Longer compositions can be obtained by nesting Seq.

~~~
Seq( e1, Seq(e2, Seq(e3, Seq(e4, e5))))
~~~

## 2 A (5 points): Extend the abstract syntax with Seq

In [6]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// New Ref
case class NewRef(e: Expr) extends Expr

//DeRef
case class DeRef(lval: Expr) extends Expr

//AssignRef
case class AssignRef(lval: Expr, rval: Expr) extends Expr

// Seq
// YOUR CODE HERE
case class Seq(e1: Expr, e2: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Geq
defined class Eq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined class Seq

In [7]:
//BEGIN TEST
val s1 = Seq(Const(10.0), NewRef(Const(15.0)))
val s2 = Seq(DeRef(NewRef(Const(20.0))), Let("x", DeRef(NewRef(Const(25.0))), Ident("x")))
val s3 = Seq(Const(10.0), Seq(Const(25.0), Const(35.0)))
val s4 = Seq(Seq(Const(5.0), Const(10.0)),  Seq(Const(25.0), Const(35.0)) )

passed(5)
//END TEST


*** Tests Passed (5 points) ***


s1: Seq = Seq(e1 = Const(v = 10.0), e2 = NewRef(e = Const(v = 15.0)))
s2: Seq = Seq(
  e1 = DeRef(lval = NewRef(e = Const(v = 20.0))),
  e2 = Let(
    s = "x",
    defExpr = DeRef(lval = NewRef(e = Const(v = 25.0))),
    bodyExpr = Ident(s = "x")
  )
)
s3: Seq = Seq(
  e1 = Const(v = 10.0),
  e2 = Seq(e1 = Const(v = 25.0), e2 = Const(v = 35.0))
)
s4: Seq = Seq(
  e1 = Seq(e1 = Const(v = 5.0), e2 = Const(v = 10.0)),
  e2 = Seq(e1 = Const(v = 25.0), e2 = Const(v = 35.0))
)

## 2 B (5 Points): Write Semantics

We would like to write a semantic rule for evaluating $\texttt{Seq(e1, e2)}$. Complete the rules for the
OK and error cases by filling in the ??? marks below.

$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma, s) = (v_1, s_1),\;\; v_1 \not= \mathbf{error},\;\;\mathbf{eval}(\texttt{e2}, \sigma, \color{red}{1} ) = (v_2, s_2)\\
\hline
\mathbf{eval}(\texttt{Seq(e1, e2)}, \sigma, s) = \color{red}{2} \\
\end{array}(\text{seq-ok})$$


$$\begin{array}{c}
\mathbf{eval}(\texttt{e1}, \sigma, s) = (v_1, s_1),\;\; v_1 = \mathbf{error} \\
\hline
\mathbf{eval}(\texttt{Seq(e1, e2)}, \sigma, s) = \color{red}{3} \\
\end{array}(\text{seq-nok})$$

Fill in the appropriate values for  $\color{red}{1}, \color{red}{2}, \color{red}{3}$.

Write your answer in the cell below. You can make a numbered list in markdown to represent your answers as follows:
1. First
2. Second
3. And so on...

1. s 
2. (v2,s2)
3. ((error, s1))

## 2 C (10 points): Implement the code

Extend the implementation of the interpreter from the notebook on references to add support for Seq.

In [8]:
sealed trait Value
/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value
/*2. Operators on values */
def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}
def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}
def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store */
case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) 
                (env(x), store)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        case NewRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            val (store2, j) = createNewCell(store1, v)
            (Reference(j), store2)
        }
        
        case DeRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val v = lookupCellValue(store1, j)
                    (v, store1)
                }
                case _ => throw new IllegalArgumentException(s"Deref applied to an expression that does not evaluate to a reference")
            }
        }
        
        case AssignRef(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case Reference(j) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    val store3 = assignToCell(store2, j, v2)
                    (v2, store3)
                }
                case _ => throw new IllegalArgumentException(s"AssignRef applied to argument that is not a reference")
                
            }
        }
        
        case Seq(e1, e2) => {
            // YOUR CODE HERE
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1) 
            (v2, store2);
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}

defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell
defined function evalExpr
defined function evalProgram

In [9]:
//BEGIN TEST

val incr = Ident("incr")
val decr = Ident("decr")
val zero = Ident("zero")
val r = Ident("r")
val x = Ident("x")
val e4 = Seq(Seq(Seq(FunCall(zero, r), FunCall(incr, r)), Seq(FunCall(incr, r), FunCall(incr, r))), Seq(FunCall(decr, r), FunCall(decr,r)))
val rr = Let("r", NewRef(Const(15.0)), e4)
val z = Let("zero", FunDef("x", AssignRef(x, Const(0.0))), rr)
val d = Let("decr", FunDef("x", AssignRef(x, Minus(DeRef(x), Const(1.0)))), z)
val i = Let("incr",FunDef("x", AssignRef(x, Plus(DeRef(x), Const(1.0)))), d )
val prog = TopLevel(i)
             
assert(evalProgram(prog) == NumValue(1.0), "Test 1 Set 3 Failed")

passed(10)
//END TEST


*** Tests Passed (10 points) ***


incr: Ident = Ident(s = "incr")
decr: Ident = Ident(s = "decr")
zero: Ident = Ident(s = "zero")
r: Ident = Ident(s = "r")
x: Ident = Ident(s = "x")
e4: Seq = Seq(
  e1 = Seq(
    e1 = Seq(
      e1 = FunCall(funCalled = Ident(s = "zero"), argExpr = Ident(s = "r")),
      e2 = FunCall(funCalled = Ident(s = "incr"), argExpr = Ident(s = "r"))
    ),
    e2 = Seq(
      e1 = FunCall(funCalled = Ident(s = "incr"), argExpr = Ident(s = "r")),
      e2 = FunCall(funCalled = Ident(s = "incr"), argExpr = Ident(s = "r"))
    )
  ),
  e2 = Seq(
    e1 = FunCall(funCalled = Ident(s = "decr"), argExpr = Ident(s = "r")),
    e2 = FunCall(funCalled = Ident(s = "decr"), argExpr = Ident(s = "r"))
  )
)
rr: Let = Let(
  s = "r",
  defExpr = NewRef(e = Const(v = 15.0)),
  bodyExpr = Seq(
    e1 = Seq(
      e1 = Seq(
        e1 = FunCall(funCalled = Ident(s = "zero"), argExpr = Ident(s = "r")),
        e2 = FunCall(funCalled = Ident(s = "incr"), argExpr = Ident(s = "r"))
      ),
      e2 = Seq(
        e

## That's All Folks!!